In [5]:
# importing required modules
import PyPDF2

# creating a pdf file object
pdfFileObj = open('ComputingScienceSummaries.pdf', 'rb')
  
# creating a pdf reader object
pdfReader = PyPDF2.PdfReader(pdfFileObj)
  
text = ''

n_pages = len(pdfReader.pages)

for n in range(n_pages):
    pageObj = pdfReader.pages[n]
    text += pageObj.extract_text()
  
# closing the pdf file object
pdfFileObj.close()

In [20]:
courses = {}
for line in text.splitlines():
    if line.startswith('CSE'):
        course = line.replace('  ', ' ').strip()
        courses[course] = []
    elif 'Prerequisite' in line:
        preq = line.split(': ')[1]
        if 'None' in preq:
            preq = ''
        preq = preq.strip()
        print(course)
        print(preq)

CSE1010: COMPUTER SCIENCE 1

CSE1110: STRUCTURED PROGRAMMING 1

CSE1110: STRUCTURED PROGRAMMING 1
CSE1110
CSE1110: STRUCTURED PROGRAMMING 1

CSE1220: CLIENT -SIDE SCRIPTING 2

CSE1240: ROBOTICS PROGRAMMING 1
CSE1110
CSE1910: CSE PROJECT A

CSE2010: COMPUTER SCIENCE 2
CSE1010
CSE2110: PROCEDURAL PROGRAMMING 1
CSE1120
CSE2110: PROCEDURAL PROGRAMMING 1
CSE2110
CSE2130: FILES & FILE STRUCTURES 1
CSE2120
CSE2140: SECOND LANGUAGE PROGRAMMING 1
CSE2110
CSE2210: CLIENT -SIDE SCRIPTING 3
CSE1220
CSE2240: ROBOTICS PROGRAMMING 2
CSE1240
CSE2240: ROBOTICS PROGRAMMING 2

CSE2920: CSE PROJECT C

CSE2950: CSE INTERMEDIATE PRACTICUM

CSE3010: COMPUTER SCIENCE 3
CSE2010
CSE3020: COMPUTER SCIENCE 4
CSE3010
CSE3110: ITERATIVE ALGORITHM 1
CSE2120
CSE3120: OBJECT -ORIENTED PROGRAMMING 1
CSE2110
CSE3130: OBJECT -ORIENTED PROGRAMMING 2
CSE3120
CSE3140: SECOND LANGUAGE PROGRAMM ING 2
CSE2210
CSE3140: SECOND LANGUAGE PROGRAMM ING 2
CSE2240
CSE3310: RECURSIVE ALGORITHMS 1
CSE3110
CSE3320: DYNAMIC DATA STRUCTURE